## Imports and settings

In [ ]:
import sys
#source code folder
sys.path.insert(0, 'C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/ADL4CV_project')
#model folder
sys.path.insert(0, 'C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/ADL4CV_project/models/')
#data folder
sys.path.insert(0, 'C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/data')

In [ ]:
import os
import numpy as np
import src
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import keras.backend as K
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.client import device_lib
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization, TimeDistributed, LSTM
from keras.optimizers import Adam, RMSprop
print(device_lib.list_local_devices())

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "0" #"" for CPU

Load dataset.

In [ ]:
past = 2
name = f"unet_{past}-1"
batch_size=64
epochs = 30

In [ ]:
train, xval, test = src.load_datasets(past_frames=past)

In [ ]:
unet_train, unet_truth, unet_val, unet_val_truth, unet_test, unet_test_truth = src.split_datasets(
            train[:2000], xval, test, past_frames=past, augment=True)

## U-net

In [ ]:
model = src.unet(input_shape=(64, 64, past), dropout=0, batchnorm=True)

In [ ]:
model.summary()

In [ ]:
#train the model
model.compile(optimizer=tf.train.AdamOptimizer(learning_rate = 0.0001),loss=src.custom_loss("l2+gdl"), metrics=[src.relative_error_tensor]) 

In [ ]:
model.fit(unet_train,
          unet_truth,
          batch_size = batch_size,
          epochs=epochs,
          validation_data=(unet_val,unet_val_truth),
          shuffle = True)

## Save features

In [ ]:
hist = model.history
hist.history.keys()
plt.plot(hist.history['loss'],)
plt.plot(hist.history['val_loss'])
plt.grid()
plt.legend(['training', 'validation'], loc='upper left')
plt.xlabel("Epoch")
plt.ylabel("Loss")
#plt.yscale("log")
plt.savefig(name+'_curves.png')

Predict future frames. Loads a 20 long sequence with 1000 sequence samples.

In [ ]:
sequence_test = src.load_datasets(prediction=True)

In [ ]:
sequence_test = src.augment_data(sequence_test[:100])

In [ ]:
sequence_test.shape

In [ ]:
#test
predictions = {}
past_frames = sequence_test[...,0:past]
test_truth = sequence_test[...,past:past+1]

In [ ]:
for t in range(5):
    future = model.predict(past_frames, batch_size=64)
    predictions[f"{t}"] = future
    past_frames = np.concatenate((past_frames[:,:,:,1:], predictions[f"{t}"]), axis=-1)
    test_truth = sequence_test[...,past+1+t:past+2+t]

Save example predictions

In [ ]:
src.save_examples(name, sequence_test, predictions, past, samples=[33]) # 33, 46, 54

Calculate scores

In [ ]:
#test
predictions = model.predict(unet_test, batch_size=50)
truth       = unet_test_truth
truth.shape

Get ordered predictions by pixelwise difference

In [ ]:
args = src.arg_getter(truth, predictions)
args[:10]

In [ ]:
error_images, error_vals, error_means = src.error_distribution(truth,predictions)

Plot and save examples

In [ ]:
src.result_plotter(args[:5], (unet_test[:,:,:,0], truth[:,:,:,0], predictions[:,:,:,0], error_images[:,:,:,0]))